# Create basic visualisation elements using Plotly on DASH 

In [ ]:
https://plotly.com/python/plotly-express/

#https://dash.gallery/Portal/

In [26]:
# Import Libraries

import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
from sklearn import datasets, linear_model
from sklearn.metrics import r2_score

### Basic Visualisations with Time Series

In [27]:
# Import data 

df = pd.read_csv('datavisnew.csv')

In [28]:
# Test data types

column_data_types = df.dtypes
print("Data Types of DataFrame Columns:")
print(column_data_types)

Data Types of DataFrame Columns:
Comment    object
Time       object
Score       int64
ID          int64
Hour        int64
dtype: object


In [29]:
# Convert df to correct datetime format

df['Time'] = pd.to_datetime(df['Time'], format='%Y-%m-%d %H:%M:%S')

In [30]:
df.head()

,Comment,Time,Score,ID,Hour
0,Posting a bit early so people know where to go...,2023-10-10 11:14:12,1,1,11
1,20% fare reduction on public transport continu...,2023-10-10 13:55:05,39,2,13
2,This is good.,2023-10-10 14:16:50,3,3,14
3,"Aaaaaahhhhh, what have the Greens done for us!",2023-10-10 16:56:05,2,4,16
4,"€67 million to provide 16,000 craft apprentice...",2023-10-10 14:18:04,25,5,14


In [31]:
column_data_types = df.dtypes
print("Data Types of DataFrame Columns:")
print(column_data_types)

Data Types of DataFrame Columns:
Comment            object
Time       datetime64[ns]
Score               int64
ID                  int64
Hour                int64
dtype: object


In [35]:
# Create Minute column for further analysys

# Check for rows with invalid datetime values
invalid_rows = df[df['Time'].isnull()]

# If there are invalid datetime values, you may want to handle them before proceeding
# For example, you can drop rows with invalid datetime values
df = df.dropna(subset=['Time'])

# Create the 'Minutes' column
df['Minutes'] = df['Time'].dt.minute


In [37]:
# Export dataframe to csv to use within dash app cells

df.to_csv('datavismins.csv', index=False)

In [36]:
df.head()

,Comment,Time,Score,ID,Hour,Minutes
0,Posting a bit early so people know where to go...,2023-10-10 11:14:12,1,1,11,14
1,20% fare reduction on public transport continu...,2023-10-10 13:55:05,39,2,13,55
2,This is good.,2023-10-10 14:16:50,3,3,14,16
3,"Aaaaaahhhhh, what have the Greens done for us!",2023-10-10 16:56:05,2,4,16,56
4,"€67 million to provide 16,000 craft apprentice...",2023-10-10 14:18:04,25,5,14,18


In [38]:
# Test visualisation 1 - Scores bar chart


# Create a bar plot of all comment scores
fig = px.bar(df, x='ID', y='Score', title='Comment Scores')

# Show the plot
fig.show()



In [44]:
# Comment Scores Visualisation Element


# Create Dash App
app = dash.Dash(__name__)

# Layout app
app.layout = html.Div([
    html.H1("Comment Scores Bar Plot"),
    
    # Add dropdown with 4 features we want to display (ID, Score, Hour, Minute)
    dcc.Dropdown(
        id='x-axis-dropdown',
        options=[
            {'label': column, 'value': column}
            for column in ['ID', 'Score', 'Hour','Minutes']  
        ],
        value='ID',  
        style={'width': '50%'}
    ),

    
    dcc.Graph(
        id='bar-chart',
        config={'displayModeBar': False}  
    )
])

# Define callback 
@app.callback(
    Output('bar-chart', 'figure'),
    [Input('x-axis-dropdown', 'value')]
)
def update_chart(selected_column):
    # Create 'Score' bar chart
    fig = px.bar(df, x=selected_column, y='Score', title=f'Comment Scores - {selected_column}', color='Score',
                 color_continuous_scale='Plasma')

    
    fig.update_layout(coloraxis_colorbar=dict(yanchor="middle", y=0.5, x=-0.25))

    return fig

# Run the app on alt port for testing
if __name__ == '__main__':
    app.run_server(debug=True, port=8051)
